In [1]:
# Install required packages
%pip install rotary-embedding-torch transformers pyyaml

# Import all dependencies
import os
import math
import time
import inspect
from dataclasses import dataclass, field
from typing import Optional

# PyTorch
import torch
import torch.nn as nn
from torch.nn import functional as F

# Transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

# YAML configuration
import yaml

# Rotary Embedding
from rotary_embedding_torch import RotaryEmbedding

print("All dependencies imported successfully!")

Note: you may need to restart the kernel to use updated packages.


c:\Users\ssuga\anaconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All dependencies imported successfully!


In [2]:
# All imports are in the first cell
# This cell is kept for backward compatibility or can be removed


In [3]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-135M")

In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
    (rotary_emb): LlamaRotaryEm

In [5]:
print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
    (rotary_emb): LlamaRotaryEm

134515008

In [6]:
# Load configuration from YAML file
# All imports (yaml, dataclass, field, Optional) are in the first cell

# Load defaults from config.yaml first
def load_yaml_defaults(yaml_path: str = "config.yaml"):
    """Load default values from YAML file"""
    with open(yaml_path, 'r') as f:
        return yaml.safe_load(f) or {}

# Load YAML to get defaults
_yaml_defaults = load_yaml_defaults("config.yaml")
_model_defaults = _yaml_defaults.get("model", {})
_tokenizer_defaults = _yaml_defaults.get("tokenizer", {})

@dataclass
class ModelConfig:
    """Model architecture and initialization parameters"""
    model_type: str = _model_defaults.get("model_type", "llama")
    vocab_size: int = _model_defaults.get("vocab_size", 49152)
    hidden_size: int = _model_defaults.get("hidden_size", 576)
    intermediate_size: int = _model_defaults.get("intermediate_size", 1536)
    num_hidden_layers: int = _model_defaults.get("num_hidden_layers", 30)
    num_attention_heads: int = _model_defaults.get("num_attention_heads", 18)
    num_key_value_heads: int = _model_defaults.get("num_key_value_heads", 6)
    max_position_embeddings: int = _model_defaults.get("max_position_embeddings", 8192)
    rms_norm_eps: float = _model_defaults.get("rms_norm_eps", 1e-05)
    use_cache: bool = _model_defaults.get("use_cache", True)
    pad_token_id: Optional[int] = _model_defaults.get("pad_token_id", None)
    bos_token_id: int = _model_defaults.get("bos_token_id", 1)
    eos_token_id: int = _model_defaults.get("eos_token_id", 2)
    
    # MLP Configuration
    mlp_bias: bool = _model_defaults.get("mlp_bias", False)
    mlp_gate_proj_features: int = _model_defaults.get("mlp_gate_proj_features", 1536)
    mlp_up_proj_features: int = _model_defaults.get("mlp_up_proj_features", 1536)
    mlp_down_proj_features: int = _model_defaults.get("mlp_down_proj_features", 576)
    
    # Activation Function
    hidden_act: str = _model_defaults.get("hidden_act", "silu")
    use_silu: bool = _model_defaults.get("use_silu", True)
    
    # Model initialization
    pretrained_model_name_or_path: str = _model_defaults.get("pretrained_model_name_or_path", "HuggingFaceTB/SmolLM2-135M")
    torch_dtype: str = _model_defaults.get("torch_dtype", "float16")
    device_map: str = _model_defaults.get("device_map", "auto")
    trust_remote_code: bool = _model_defaults.get("trust_remote_code", False)

@dataclass
class TokenizerConfig:
    """Tokenizer configuration parameters"""
    tokenizer_name_or_path: str = _tokenizer_defaults.get("tokenizer_name_or_path", "HuggingFaceTB/SmolLM2-135M")
    use_fast: bool = _tokenizer_defaults.get("use_fast", True)
    padding_side: str = _tokenizer_defaults.get("padding_side", "right")
    truncation_side: str = _tokenizer_defaults.get("truncation_side", "right")

@dataclass
class SmolLMV2Config:
    """Main configuration class"""
    model: ModelConfig = field(default_factory=ModelConfig)
    tokenizer: TokenizerConfig = field(default_factory=TokenizerConfig)
    
    @classmethod
    def from_yaml(cls, yaml_path: str = "config.yaml"):
        """Load configuration from YAML file"""
        with open(yaml_path, 'r') as f:
            config_dict = yaml.safe_load(f) or {}
        
        # Create configs with defaults from YAML, then update from YAML
        model_config = ModelConfig()
        if "model" in config_dict:
            for key, value in config_dict["model"].items():
                if hasattr(model_config, key):
                    setattr(model_config, key, value)
        
        tokenizer_config = TokenizerConfig()
        if "tokenizer" in config_dict:
            for key, value in config_dict["tokenizer"].items():
                if hasattr(tokenizer_config, key):
                    setattr(tokenizer_config, key, value)
        
        return cls(model=model_config, tokenizer=tokenizer_config)

# Load configuration
cfg = SmolLMV2Config.from_yaml("config.yaml")

# Display loaded configuration
print("=" * 60)
print("Loaded SmolLMV2 Configuration")
print("=" * 60)
print("\n[Model Configuration]")
for key, value in cfg.model.__dict__.items():
    print(f"  {key}: {value}")
print("\n[Tokenizer Configuration]")
for key, value in cfg.tokenizer.__dict__.items():
    print(f"  {key}: {value}")
print("=" * 60)


Loaded SmolLMV2 Configuration

[Model Configuration]
  model_type: llama
  vocab_size: 49152
  hidden_size: 576
  intermediate_size: 1536
  num_hidden_layers: 30
  num_attention_heads: 18
  num_key_value_heads: 6
  max_position_embeddings: 8192
  rms_norm_eps: 1e-05
  use_cache: True
  pad_token_id: None
  bos_token_id: 1
  eos_token_id: 2
  mlp_bias: False
  mlp_gate_proj_features: 1536
  mlp_up_proj_features: 1536
  mlp_down_proj_features: 576
  hidden_act: silu
  use_silu: True
  pretrained_model_name_or_path: HuggingFaceTB/SmolLM2-135M
  torch_dtype: float16
  device_map: auto
  trust_remote_code: False

[Tokenizer Configuration]
  tokenizer_name_or_path: HuggingFaceTB/SmolLM2-135M
  use_fast: True
  padding_side: right
  truncation_side: right


In [7]:
class LlamaAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.num_key_value_heads = config.num_key_value_heads
        self.head_dim = config.n_embd // config.n_head
        
        # Separate projections for q, k, v, o (LLaMA style)
        # q_proj: all heads, k_proj and v_proj: only num_key_value_heads (GQA)
        self.q_proj = nn.Linear(config.n_embd, config.n_embd, bias=False)
        self.k_proj = nn.Linear(config.n_embd, config.num_key_value_heads * self.head_dim, bias=False)
        self.v_proj = nn.Linear(config.n_embd, config.num_key_value_heads * self.head_dim, bias=False)
        self.o_proj = nn.Linear(config.n_embd, config.n_embd, bias=False)
        
        # Rotary positional embedding (RoPE) - applied to q and k in forward
        self.rotary_emb = RotaryEmbedding(self.head_dim)
        
        # Causal mask
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()  # batch size, sequence length, embedding dimensionality
        
        # Project to q, k, v
        q = self.q_proj(x)  # (B, T, n_embd)
        k = self.k_proj(x)  # (B, T, num_key_value_heads * head_dim)
        v = self.v_proj(x)  # (B, T, num_key_value_heads * head_dim)
        
        # Reshape for multi-head attention
        q = q.view(B, T, self.n_head, self.head_dim).transpose(1, 2)  # (B, n_head, T, head_dim)
        k = k.view(B, T, self.num_key_value_heads, self.head_dim).transpose(1, 2)  # (B, num_key_value_heads, T, head_dim)
        v = v.view(B, T, self.num_key_value_heads, self.head_dim).transpose(1, 2)  # (B, num_key_value_heads, T, head_dim)
        
        # Apply rotary positional embeddings to q and k (RoPE)
        q = self.rotary_emb.rotate_queries_or_keys(q)
        k = self.rotary_emb.rotate_queries_or_keys(k)
        
        # Repeat k and v for GQA (if num_key_value_heads < n_head)
        if self.num_key_value_heads != self.n_head:
            # Repeat k and v to match number of query heads
            repeat_factor = self.n_head // self.num_key_value_heads
            k = k.repeat_interleave(repeat_factor, dim=1)  # (B, n_head, T, head_dim)
            v = v.repeat_interleave(repeat_factor, dim=1)  # (B, n_head, T, head_dim)
        
        # Causal self-attention: (B, n_head, T, head_dim) @ (B, n_head, head_dim, T) -> (B, n_head, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(self.head_dim))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v  # (B, n_head, T, T) @ (B, n_head, T, head_dim) -> (B, n_head, T, head_dim)

        # Re-assemble all head outputs: (B, n_head, T, head_dim) -> (B, T, C)
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        
        # Output projection
        y = self.o_proj(y)
        return y

In [8]:
class LlamaMLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.gate_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=config.mlp_bias)
        self.up_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=config.mlp_bias)
        self.down_proj = nn.Linear(config.intermediate_size, config.hidden_size, bias=config.mlp_bias)
        
        # Use activation from config.yaml
        if config.hidden_act == "silu":
            self.act_fn = nn.SiLU()
        elif config.hidden_act == "gelu":
            self.act_fn = nn.GELU()
        elif config.hidden_act == "relu":
            self.act_fn = nn.ReLU()
    
    def forward(self, x):
        # LLaMA MLP: SiLU(gate_proj(x)) * up_proj(x)
        gate = self.act_fn(self.gate_proj(x))
        up = self.up_proj(x)
        return self.down_proj(gate * up)

In [9]:
class LlamaRMSNorm(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(config.hidden_size))
        # Ensure rms_norm_eps is a float (YAML might load it as string)
        self.variance_epsilon = float(config.rms_norm_eps)
    
    def forward(self, x):
        # LLaMA RMSNorm: sqrt(1/n * sum(x^2)) * weight
        return self.weight * x / torch.sqrt(torch.mean(x * x, dim=-1, keepdim=True) + self.variance_epsilon)
    
    def __repr__(self):
        return f"LlamaRMSNorm(hidden_size={self.weight.shape[0]}, eps={self.variance_epsilon})"

In [10]:
class LlamaDecoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = LlamaRMSNorm(config)
        self.attn = LlamaAttention(config)
        self.ln_2 = LlamaRMSNorm(config)
        self.mlp = LlamaMLP(config) 

    def forward(self, x):
        # LLaMA decoder layer: ln1(x) + attn(ln1(x)) + ln2(x) + mlp(ln2(x))
        residual = x
        x = self.ln_1(x)
        x = x + self.attn(x)
        x = self.ln_2(x)
        x = x + self.mlp(x)
        x = x + residual
        return x

class LlamaDecoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.layers = nn.ModuleList([LlamaDecoderLayer(config) for _ in range(config.num_hidden_layers)])
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

class LlamaModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embed_tokens = nn.Embedding(config.vocab_size, config.n_embd)
        self.pos_embed = RotaryEmbedding(config.n_embd)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.lm_head.weight = self.embed_tokens.weight
        self.ln_f = LlamaRMSNorm(config)
        self.transformer = LlamaDecoder(config)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std = 0.02)
            
    def forward(self, idx, targets=None):
        # idx is of shape (B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        # forward the token embeddings (RoPE is applied in attention, not here)
        x = self.embed_tokens(idx)  # token embeddings of shape (B, T, n_embd)
        # forward the blocks of the transformer
        x = self.transformer(x)
        # forward the final layernorm and the classifier
        x = self.ln_f(x)
        logits = self.lm_head(x)  # (B, T, vocab_size)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss




    def generate(self, input_ids, max_new_tokens):
        """
        Generate new tokens up to max_new_tokens.
        
        Args:
            input_ids: Input token IDs, shape (batch_size, seq_len)
            max_new_tokens: Maximum number of new tokens to generate
            
        Returns:
            Generated token IDs, shape (batch_size, seq_len + max_new_tokens)
        """
        self.eval()  # Set to evaluation mode
        input_ids = input_ids.to(self.config.device)
        
        # Start with the input sequence
        generated_ids = input_ids.clone()
        
        # Generate tokens one at a time
        for _ in range(max_new_tokens):
            # Get embeddings for current sequence
            x = self.embed_tokens(generated_ids)
            # Note: Rotary embeddings (RoPE) are applied in attention, not here
            
            # Pass through transformer
            x = self.transformer(x)
            x = self.ln_f(x)
            
            # Get logits for all positions
            logits = self.lm_head(x)
            
            # Only use the last token's logits to predict next token
            logits = logits[:, -1, :]  # Shape: (batch_size, vocab_size)
            
            # Greedy decoding: pick the token with highest probability
            next_token = torch.argmax(logits, dim=-1, keepdim=True)  # Shape: (batch_size, 1)
            
            # Append the new token to the sequence
            generated_ids = torch.cat([generated_ids, next_token], dim=1)
        
        # Return all generated tokens (input + newly generated)
        return generated_ids
    

    
    

In [11]:
# Detailed Step-by-Step Explanation of generate() method
import torch

print("=" * 80)
print("STEP-BY-STEP EXPLANATION OF GENERATE METHOD")
print("=" * 80)
print()

# ============================================================================
# STEP 1: Input Preparation
# ============================================================================
print("STEP 1: Input Preparation")
print("-" * 80)

# Example: We want to generate text starting from "The weather is"
# After tokenization, this becomes token IDs
input_ids = torch.tensor([[1234, 5678, 9012]])  # Shape: (batch_size=1, sequence_length=3)
print(f"input_ids shape: {input_ids.shape}")
print(f"input_ids: {input_ids}")
print("  - batch_size = 1 (one sequence)")
print("  - sequence_length = 3 (three tokens: 'The', 'weather', 'is')")
print()

# ============================================================================
# STEP 2: Embedding Layer
# ============================================================================
print("STEP 2: Embedding Layer (embed_tokens)")
print("-" * 80)

# Each token ID is converted to a dense vector
# vocab_size = 49152, hidden_size = 576
# embed_tokens converts token IDs to embeddings
batch_size, seq_len = input_ids.shape
hidden_size = 576

# Simulate embedding: (batch, seq_len) -> (batch, seq_len, hidden_size)
x = torch.randn(batch_size, seq_len, hidden_size)  # Simulated embeddings
print(f"After embedding, x shape: {x.shape}")
print(f"  - batch_size = {batch_size}")
print(f"  - sequence_length = {seq_len}")
print(f"  - hidden_size = {hidden_size}")
print("  - Each token is now a 576-dimensional vector")
print()

# ============================================================================
# STEP 3: Positional Embedding
# ============================================================================
print("STEP 3: Positional Embedding (pos_embed)")
print("-" * 80)

# Add positional information to embeddings
pos_embed = torch.randn(batch_size, seq_len, hidden_size)  # Simulated
x = x + pos_embed
print(f"After positional embedding, x shape: {x.shape}")
print("  - Still same shape: (batch, seq_len, hidden_size)")
print("  - Now each token has both semantic and positional information")
print()

# ============================================================================
# STEP 4: Transformer Layers
# ============================================================================
print("STEP 4: Transformer Layers (transformer)")
print("-" * 80)

# Pass through all transformer decoder layers
# Each layer processes the entire sequence
x = torch.randn(batch_size, seq_len, hidden_size)  # Simulated after transformer
print(f"After transformer, x shape: {x.shape}")
print("  - Still same shape: (batch, seq_len, hidden_size)")
print("  - Each position now has contextualized representations")
print("  - Token 'is' now knows about 'The' and 'weather'")
print()

# ============================================================================
# STEP 5: Final Layer Norm
# ============================================================================
print("STEP 5: Final Layer Normalization (ln_f)")
print("-" * 80)

x = torch.randn(batch_size, seq_len, hidden_size)  # Simulated after norm
print(f"After layer norm, x shape: {x.shape}")
print("  - Still same shape: (batch, seq_len, hidden_size)")
print("  - Normalized for stability")
print()

# ============================================================================
# STEP 6: Language Model Head (lm_head)
# ============================================================================
print("STEP 6: Language Model Head (lm_head)")
print("-" * 80)

# Convert hidden states to vocabulary logits
vocab_size = 49152
logits = torch.randn(batch_size, seq_len, vocab_size)  # Simulated logits
print(f"After lm_head, logits shape: {logits.shape}")
print(f"  - batch_size = {batch_size}")
print(f"  - sequence_length = {seq_len}")
print(f"  - vocab_size = {vocab_size}")
print()
print("What this means:")
print("  - For EACH position in the sequence, we have predictions for ALL tokens")
print("  - Position 0 ('The'): 49152 logits (probabilities for next token)")
print("  - Position 1 ('weather'): 49152 logits (probabilities for next token)")
print("  - Position 2 ('is'): 49152 logits (probabilities for next token)")
print()
print("Example logits for position 2 ('is'):")
example_logits = logits[0, 2, :10]  # First 10 tokens
print(f"  First 10 logits: {example_logits}")
print("  - These are raw scores (not probabilities yet)")
print("  - Higher values = more likely token")
print()

# ============================================================================
# STEP 7: Extract Last Token Logits (THE KEY LINE!)
# ============================================================================
print("STEP 7: Extract Last Token Logits - logits[:, -1, :]")
print("-" * 80)
print("This is the line you asked about!")
print()

print("BEFORE slicing:")
print(f"  logits.shape = {logits.shape}  # (batch, seq_len, vocab_size)")
print(f"  logits = {logits.shape[0]} batch × {logits.shape[1]} positions × {logits.shape[2]} vocab tokens")
print()

print("Slicing operation: logits[:, -1, :]")
print("  - ':' in first position = keep ALL batches")
print("  - '-1' in second position = take LAST position (last token)")
print("  - ':' in third position = keep ALL vocabulary tokens")
print()

logits_last = logits[:, -1, :]
print("AFTER slicing:")
print(f"  logits_last.shape = {logits_last.shape}  # (batch, vocab_size)")
print(f"  logits_last = {logits_last.shape[0]} batch × {logits_last.shape[1]} vocab tokens")
print()

print("What we kept:")
print("  - We ONLY kept the logits from the LAST token position ('is')")
print("  - We DISCARDED logits from positions 0 ('The') and 1 ('weather')")
print("  - Why? Because we only need the LAST token to predict the NEXT token!")
print()

# ============================================================================
# STEP 8: Get Next Token (Greedy Decoding)
# ============================================================================
print("STEP 8: Get Next Token (torch.argmax)")
print("-" * 80)

# Find the token with highest logit (greedy decoding)
next_token = torch.argmax(logits_last, dim=-1, keepdim=True)
print(f"next_token shape: {next_token.shape}")
print(f"next_token: {next_token}")
print()
print("What happened:")
print("  - torch.argmax finds the index with the highest value")
print("  - dim=-1 means we search along the vocabulary dimension")
print("  - keepdim=True keeps the dimension for concatenation")
print("  - Result: The token ID of the most likely next token")
print()

# ============================================================================
# VISUAL SUMMARY
# ============================================================================
print("=" * 80)
print("VISUAL SUMMARY")
print("=" * 80)
print()
print("Input sequence: ['The', 'weather', 'is']")
print()
print("After forward pass:")
print("  logits shape: (1, 3, 49152)")
print("  ┌─────────────────────────────────────┐")
print("  │ Position 0 ('The'):    49152 logits │")
print("  │ Position 1 ('weather'): 49152 logits │")
print("  │ Position 2 ('is'):     49152 logits │ ← We need THIS one!")
print("  └─────────────────────────────────────┘")
print()
print("After logits[:, -1, :]:")
print("  logits shape: (1, 49152)")
print("  ┌─────────────────────┐")
print("  │ Position 2 ('is'):  │ ← Only the last token's logits")
print("  │   49152 logits      │")
print("  └─────────────────────┘")
print()
print("After argmax:")
print("  next_token: [token_id]  ← The most likely next token")
print()
print("=" * 80)


STEP-BY-STEP EXPLANATION OF GENERATE METHOD

STEP 1: Input Preparation
--------------------------------------------------------------------------------
input_ids shape: torch.Size([1, 3])
input_ids: tensor([[1234, 5678, 9012]])
  - batch_size = 1 (one sequence)
  - sequence_length = 3 (three tokens: 'The', 'weather', 'is')

STEP 2: Embedding Layer (embed_tokens)
--------------------------------------------------------------------------------
After embedding, x shape: torch.Size([1, 3, 576])
  - batch_size = 1
  - sequence_length = 3
  - hidden_size = 576
  - Each token is now a 576-dimensional vector

STEP 3: Positional Embedding (pos_embed)
--------------------------------------------------------------------------------
After positional embedding, x shape: torch.Size([1, 3, 576])
  - Still same shape: (batch, seq_len, hidden_size)
  - Now each token has both semantic and positional information

STEP 4: Transformer Layers (transformer)
--------------------------------------------------

In [12]:

# Use tokenizer from config.yaml (loaded in cell 2 or 4)
# If not available, load it from config
if 'tokenizer' not in globals():
    with open("config.yaml", 'r') as f:
        config_dict = yaml.safe_load(f)
    tokenizer = AutoTokenizer.from_pretrained(
        config_dict["tokenizer"]["tokenizer_name_or_path"],
        use_fast=config_dict["tokenizer"]["use_fast"]
    )

# Read and tokenize input text
with open('input.txt', 'r') as f:
    text = f.read()

# Limit text and tokenize using tokenizer from config.yaml
text = text[:1000]
tokens = tokenizer.encode(text, add_special_tokens=False)  # Returns list of token IDs

# Setup batch and sequence parameters
B, T = 4, 32  # batch_size, sequence_length

# Create input/target pairs from tokens
buf = torch.tensor(tokens[:B*T + 1])  # Take B*T+1 tokens for input/target split
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
buf = buf.to(device)

# Split into input (x) and target (y) - y is x shifted by 1
x = buf[:-1].view(B, T)  # Shape: (B, T)
y = buf[1:].view(B, T)   # Shape: (B, T)

# Prepare model config from config.yaml
# Add compatibility attributes for LlamaModel/LlamaAttention
model_config = cfg.model
model_config.n_embd = model_config.hidden_size
model_config.n_head = model_config.num_attention_heads
model_config.block_size = model_config.max_position_embeddings
model_config.device = device
model_config.num_key_value_heads = model_config.num_key_value_heads

# Create LlamaModel using config from config.yaml
model = LlamaModel(model_config)
model.to(device)

# Forward pass and compute loss
logits, loss = model(x, y)
print(f"Loss: {loss.item():.4f}") 

Loss: 11.2363


In [13]:
# Calculate expected loss for random prediction (uniform distribution)
# This represents the worst-case scenario: predicting uniformly at random
# Formula: -log(1/vocab_size) = log(vocab_size)
vocab_size = cfg.model.vocab_size
random_loss = -torch.log(torch.tensor(1.0 / vocab_size))
print(f"Expected loss for random prediction (vocab_size={vocab_size}): {random_loss.item():.4f}")
print(f"This is the theoretical maximum loss when predicting uniformly at random.")

Expected loss for random prediction (vocab_size=49152): 10.8027
This is the theoretical maximum loss when predicting uniformly at random.


In [14]:
#Explaining the model 
model

LlamaModel(
  (embed_tokens): Embedding(49152, 576)
  (pos_embed): RotaryEmbedding()
  (lm_head): Linear(in_features=576, out_features=49152, bias=False)
  (ln_f): LlamaRMSNorm(hidden_size=576, eps=1e-05)
  (transformer): LlamaDecoder(
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (ln_1): LlamaRMSNorm(hidden_size=576, eps=1e-05)
        (attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (ln_2): LlamaRMSNorm(hidden_size=576, eps=1e-05)
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Line

In [15]:
print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(model)

LlamaModel(
  (embed_tokens): Embedding(49152, 576)
  (pos_embed): RotaryEmbedding()
  (lm_head): Linear(in_features=576, out_features=49152, bias=False)
  (ln_f): LlamaRMSNorm(hidden_size=576, eps=1e-05)
  (transformer): LlamaDecoder(
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (ln_1): LlamaRMSNorm(hidden_size=576, eps=1e-05)
        (attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (ln_2): LlamaRMSNorm(hidden_size=576, eps=1e-05)
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Line

134515008

In [ ]:
# Device selection
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"Using device: {device}")

# Set seed for reproducibility
torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

# Training hyperparameters
B = 8  # batch size
T = 1024  # sequence length
num_steps = 1  # number of training steps
learning_rate = 3e-4

# DataLoader class (similar to S13.ipynb but using tokenizer from config)
class DataLoaderLite:
    def __init__(self, B, T, tokenizer):
        self.B = B
        self.T = T
        self.tokenizer = tokenizer

        # Load tokens from disk and store them in memory
        with open('input.txt', 'r') as f:
            text = f.read()
        
        # Tokenize using the tokenizer from config.yaml
        tokens = tokenizer.encode(text, add_special_tokens=False)
        self.tokens = torch.tensor(tokens)
        print(f'Loaded {len(self.tokens)} tokens')
        print(f'1 epoch = {len(self.tokens) // (B * T)} batches')

        # State
        self.current_position = 0
    
    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position: self.current_position + B * T + 1]
        x = (buf[:-1]).view(B, T)  # inputs
        y = (buf[1:]).view(B, T)  # targets
        # Advance the position in the tensor
        self.current_position += B * T
        # If loading the next batch would be out of bounds, reset
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.current_position = 0
        return x, y

# Ensure tokenizer is available (should be loaded from config.yaml in earlier cells)
if 'tokenizer' not in globals():
    with open("config.yaml", 'r') as f:
        config_dict = yaml.safe_load(f)
    tokenizer = AutoTokenizer.from_pretrained(
        config_dict["tokenizer"]["tokenizer_name_or_path"],
        use_fast=config_dict["tokenizer"]["use_fast"]
    )

# Create data loader
train_loader = DataLoaderLite(B=B, T=T, tokenizer=tokenizer)

# Ensure model exists and is on correct device
if 'model' not in globals():
    # Create model config if needed
    model_config = cfg.model
    model_config.n_embd = model_config.hidden_size
    model_config.n_head = model_config.num_attention_heads
    model_config.block_size = model_config.max_position_embeddings
    model_config.device = device
    model_config.num_key_value_heads = model_config.num_key_value_heads
    
    # Create LlamaModel
    model = LlamaModel(model_config)
    model.to(device)
else:
    model = model.to(device)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop with profiling
print("\n" + "="*80)
print("Starting Training with Profiling")
print("="*80)
print(f"Batch size: {B}, Sequence length: {T}, Steps: {num_steps}")
print("="*80 + "\n")

# Setup profiler
profiler_activities = [torch.profiler.ProfilerActivity.CPU]
if torch.cuda.is_available():
    profiler_activities.append(torch.profiler.ProfilerActivity.CUDA)

# Create log directory if it doesn't exist
os.makedirs('./log/profiler', exist_ok=True)

with torch.profiler.profile(
    activities=profiler_activities,
    schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
    on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/profiler'),
    record_shapes=True,
    profile_memory=True,
    with_stack=True
) as prof:
    
    for step in range(num_steps):
        t0 = time.time()
        
        # Get batch
        x, y = train_loader.next_batch()
        x, y = x.to(device), y.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        logits, loss = model(x, y)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Synchronize for accurate timing
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        
        t1 = time.time()
        dt = (t1 - t0) * 1000  # milliseconds
        tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
        
        # Step profiler
        prof.step()
        
        # Print metrics
        print(f'Step {step:2d} | Loss: {loss.item():.4f} | dt: {dt:6.2f}ms | tok/sec: {tokens_per_sec:8.2f}')

print("\n" + "="*80)
print("Training Complete!")
print("="*80)
print(f"Final loss: {loss.item():.4f}")

# Print profiler summary
print("\n" + "="*80)
print("Profiler Summary")
print("="*80)
print(prof.key_averages().table(sort_by="cuda_time_total" if torch.cuda.is_available() else "cpu_time_total", row_limit=20))

# Export profiler trace (for TensorBoard)
print("\nProfiler trace saved to: ./log/profiler")
print("View with: tensorboard --logdir=./log/profiler")


Using device: cpu


Token indices sequence length is longer than the specified maximum sequence length for this model (341094 > 8192). Running this sequence through the model will result in indexing errors


Loaded 341094 tokens
1 epoch = 41 batches

Starting Training with Profiling
Batch size: 8, Sequence length: 1024, Steps: 10

